In [ ]:
import os
from pathlib import Path

import pandas as pd
from us.states import NY

# Change working directory to project folder
old_cwd = os.getcwd()
PROJECT_DIRECTORY = Path().resolve().parent
os.chdir(PROJECT_DIRECTORY)
print(f"Changed working directory from {old_cwd} to {os.getcwd()}")

In [ ]:
from python.geospatial import (
    NYC_PROJECTION,
    WKT_PROJECTION,
    map_folium,
    map_simple,
    reproject_geometry,
)
from python.utils import (
    load_geodata_url,
    load_shapefile,
)

pd.options.display.max_rows = 20

DATA_DIRECTORY = ".data/inspect_shapefile"
CENSUS_SHAPEFILE_URL_COUNTY = (
    "https://www2.census.gov/geo/tiger/TIGER2020/COUNTY/tl_2020_us_county.zip"
)
CENSUS_SHAPEFILE_URL_PUMA_NY = (
    f"https://www2.census.gov/geo/tiger/TIGER2020/PUMA20/tl_2020_{NY.fips}_puma20.zip"
)

## Census data

In [ ]:
census_pumas_ny = load_geodata_url(CENSUS_SHAPEFILE_URL_PUMA_NY)
census_pumas_ny.crs

In [ ]:
census_pumas_ny

In [ ]:
census_pumas_nyc = census_pumas_ny[census_pumas_ny["NAMELSAD20"].str.contains("NYC")]
census_pumas_nyc = census_pumas_nyc.to_crs(NYC_PROJECTION)

In [ ]:
map_simple(census_pumas_nyc)

In [ ]:
census_counties = load_geodata_url(CENSUS_SHAPEFILE_URL_COUNTY)
census_counties.crs

In [ ]:
census_counties

In [ ]:
census_counties_ny = census_counties[census_counties["STATEFP"] == NY.fips]
census_counties_nyc = census_counties_ny[census_counties_ny["CLASSFP"] == "H6"]
census_counties_nyc = census_counties_nyc.to_crs(NYC_PROJECTION)
census_counties_nyc

In [ ]:
census_counties_nyc.crs

In [ ]:
map_simple(census_counties_nyc)

In [ ]:
map_folium(census_counties_nyc)

## NYC Open Streets

In [ ]:
open_streets_geojson = load_geodata_url(
    "https://data.cityofnewyork.us/resource/uiay-nctu.geojson"
)
open_streets_geojson.geometry.crs

In [ ]:
open_streets_geojson

In [ ]:
# the CRS of the GeoDataFrame is WKT_PROJECTION
# but the CRS of the geometries is actually NYC_PROJECTION
open_streets_geojson.crs = NYC_PROJECTION
open_streets_geojson.crs

In [ ]:
open_streets_geojson

In [ ]:
map_simple(open_streets_geojson)

In [ ]:
open_streets_bk = open_streets_geojson[
    open_streets_geojson["boroughname"] == "Brooklyn"
]
open_streets_bk

In [ ]:
map_simple(open_streets_bk)

In [ ]:
map_folium(open_streets_bk)

In [ ]:
# open_streets_bk_hart = open_streets_bk.filter("appronstre")
open_streets_bk_focus = open_streets_bk[
    open_streets_bk["apprfromst"].str.contains("HALSEY")
]
open_streets_bk_focus

In [ ]:
map_folium(open_streets_bk_focus)

## CBBR

In [ ]:
data_cbbr = load_shapefile(f"{DATA_DIRECTORY}/cbbr_submissions_poly_shapefile.zip")
data_cbbr.crs

In [ ]:
data_cbbr

In [ ]:
data_cbbr.info()

In [ ]:
# count geometry types
data_cbbr.geom_type.value_counts()

In [ ]:
map_simple(data_cbbr, projection=WKT_PROJECTION)

In [ ]:
map_folium(
    data_cbbr,
    map_config={
        "color": "#e28743",
        "tiles": "CartoDB positron",
        "style_kwds": {
            "weight": 4,
        },
    },
)

## MapZAP

In [ ]:
filepath = "./.data/mapzap/mapzap_wkt_20230601.csv"
geometry_column = "wkt"
geography = load_shapefile("./.data/mapzap/mapzap_wkt_20230601.csv")

mapzap = geography
type(mapzap)

In [ ]:
mapzap.crs

In [ ]:
mapzap

In [ ]:
mapzap = reproject_geometry(mapzap, WKT_PROJECTION, NYC_PROJECTION)

In [ ]:
mapzap

In [ ]:
map_simple(mapzap)

In [ ]:
mapzap_ceqr_type_ii = mapzap[mapzap["ceqr_type"] == "Type II"]
map_simple(mapzap_ceqr_type_ii)

In [ ]:
type(mapzap_ceqr_type_ii)

In [ ]:
mapzap_ceqr_type_ii

In [ ]:
mapzap_ceqr_type_ii_for_folium = mapzap_ceqr_type_ii.dropna(subset=["geometry"])
map_folium(mapzap_ceqr_type_ii_for_folium)